# Listas de Acuerdos

Antes de ejecutar lea README.md y siga los pasos descritos.

In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
import requests

In [ ]:
# Hola a Plataforma Web API OAuth2
HOST = "https://plataforma-web-api-oauth2-dot-pjecz-268521.uc.r.appspot.com"
respuesta = requests.get(HOST)
print(respuesta.status_code)
print(respuesta.text)

In [ ]:
# Cargar username/password
load_dotenv()
parametros = {
    "username": os.getenv("USERNAME"),
    "password": os.getenv("PASSWORD"),
}
print(parametros["username"])

In [ ]:
# Solicitar token
cabeceras_para_autentificar = {'content-type': "application/x-www-form-urlencoded"}
respuesta = requests.post(f"{HOST}/token", data=parametros, headers=cabeceras_para_autentificar)
print(respuesta.status_code)

In [ ]:
# Definir cabeceras_para_api
cabeceras_para_api = None
if respuesta.status_code == 200:
    token = respuesta.json()['access_token']
    cabeceras_para_api = {'Authorization': 'Bearer ' + token}
print(cabeceras_para_api)

In [ ]:
# Función consultar
def consultar(ruta: str, parametros: dict = None):
    try:
        respuesta = requests.get(f"{HOST}/v1/{ruta}", headers=cabeceras_para_api, params=parametros, timeout=12)
    except requests.exceptions.RequestException as error:
        raise Exception("Parece que hay un error en la consulta: " + str(error))
    print(respuesta.status_code)
    if respuesta.status_code != 200:
        raise requests.HTTPError("No se tiene una respuesta con éxito")
    datos = respuesta.json()
    if "items" in datos:
        return pd.json_normalize(datos["items"])
    return pd.json_normalize(datos)

In [ ]:
# Distritos
consultar("distritos")

In [ ]:
# Autoridades
DISTRITO_ID = 8 # Distrito Judicial de Torreon (8)
MATERIA_ID = 6 # FAMILIAR ORAL (6)
parametros = {"distrito_id": DISTRITO_ID, "materia_id": MATERIA_ID}
consultar("autoridades", parametros)

In [ ]:
# Listas de Acuerdos
FECHA = "2021-09-24"
AUTORIDAD_ID = 63 # Torreon J. Segundo Familiar Oral
#parametros = {"distrito_id": DISTRITO_ID, "fecha": FECHA}
parametros = {"autoridad_id": AUTORIDAD_ID, "fecha": FECHA}
listas_de_acuerdos = consultar("listas_de_acuerdos", parametros)
listas_de_acuerdos